In [1]:
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

path = Path("./Resources/market.sqlite")
engine = create_engine(f"sqlite:///{path}")
connection = engine.connect()

In [2]:
# Read the 'market' table into a DataFrame
market_data = pd.read_sql('market', connection)

# Check data types
print(market_data.dtypes)

?                    object
Date                 object
Natural_Gas_Price    object
Natural_Gas_Vol.     object
Crude_oil_Price      object
Crude_oil_Vol.       object
Copper_Price         object
Copper_Vol.          object
Bitcoin_Price        object
Bitcoin_Vol.         object
Platinum_Price       object
Platinum_Vol.        object
Ethereum_Price       object
Ethereum_Vol.        object
S&P_500_Price        object
Nasdaq_100_Price     object
Nasdaq_100_Vol.      object
Apple_Price          object
Apple_Vol.           object
Tesla_Price          object
Tesla_Vol.           object
Microsoft_Price      object
Microsoft_Vol.       object
Silver_Price         object
Silver_Vol.          object
Google_Price         object
Google_Vol.          object
Nvidia_Price         object
Nvidia_Vol.          object
Berkshire_Price      object
Berkshire_Vol.       object
Netflix_Price        object
Netflix_Vol.         object
Amazon_Price         object
Amazon_Vol.          object
Meta_Price          

In [3]:
# Columns to be included in the targeted_data DataFrame
columns = ['Gold_Price', 'Gold_Vol.', 'Ethereum_Price', 'Ethereum_Vol.',
           'Copper_Price', 'Copper_Vol.', 'Silver_Price', 'Silver_Vol.', 
           'Bitcoin_Price', 'Bitcoin_Vol.']

# Extracting the specific columns
targeted_data = market_data[columns]


for column in targeted_data.columns:
    print(f"First 5 values of column '{column}':")
    print(market_data[column].head())
    print()  # This adds an empty line for better readability

First 5 values of column 'Gold_Price':
0    2,053.70
1    2,071.10
2    2,067.40
3    2,050.90
4    2,034.90
Name: Gold_Price, dtype: object

First 5 values of column 'Gold_Vol.':
0                      
1    260920.00000000003
2              238370.0
3              214590.0
4                1780.0
Name: Gold_Vol., dtype: object

First 5 values of column 'Ethereum_Price':
0    2,309.28
1    2,304.28
2    2,283.14
3    2,343.11
4    2,317.79
Name: Ethereum_Price, dtype: object

First 5 values of column 'Ethereum_Vol.':
0    246890.0
1    323610.0
2    408790.0
3    387120.0
4    318840.0
Name: Ethereum_Vol., dtype: object

First 5 values of column 'Copper_Price':
0    3.8215
1    3.8535
2     3.906
3     3.911
4     3.879
Name: Copper_Price, dtype: object

First 5 values of column 'Copper_Vol.':
0    
1    
2    
3    
4    
Name: Copper_Vol., dtype: object

First 5 values of column 'Silver_Price':
0    22.796
1    23.236
2    23.169
3    23.225
4    23.134
Name: Silver_Price, dtype: ob

In [4]:
# First, explicitly create a new DataFrame to avoid setting values on a slice
targeted_data = market_data[columns].copy()

# Converting price columns from string to float
targeted_data['Gold_Price'] = targeted_data['Gold_Price'].str.replace(',', '').astype(float)
targeted_data['Ethereum_Price'] = targeted_data['Ethereum_Price'].str.replace(',', '').astype(float)
targeted_data['Silver_Price'] = targeted_data['Silver_Price'].str.replace(',', '').astype(float)
targeted_data['Bitcoin_Price'] = targeted_data['Bitcoin_Price'].str.replace(',', '').astype(float)

# Replace empty strings with NaN
targeted_data.replace('', np.nan, inplace=True)

# Convert volume columns to numeric (float)
columns_to_convert = ['Gold_Vol.', 'Ethereum_Vol.', 'Copper_Vol.', 'Silver_Vol.', 'Bitcoin_Vol.']
for column in columns_to_convert:
    targeted_data.loc[:, column] = pd.to_numeric(targeted_data[column], errors='coerce')

# Handle NaN values
targeted_data.fillna(0, inplace=True)

# Check the cleaned data
print(targeted_data.head())


   Gold_Price  Gold_Vol.  Ethereum_Price  Ethereum_Vol. Copper_Price  \
0      2053.7        0.0         2309.28       246890.0       3.8215   
1      2071.1   260920.0         2304.28       323610.0       3.8535   
2      2067.4   238370.0         2283.14       408790.0        3.906   
3      2050.9   214590.0         2343.11       387120.0        3.911   
4      2034.9     1780.0         2317.79       318840.0        3.879   

   Copper_Vol.  Silver_Price  Silver_Vol.  Bitcoin_Price Bitcoin_Vol.  
0          0.0        22.796          0.0        43194.7      42650.0  
1          0.0        23.236      85160.0        43081.4      47690.0  
2          0.0        23.169      66910.0        42580.5      56480.0  
3          0.0        23.225      53370.0        42946.2      55130.0  
4          0.0        23.134        330.0        43299.8      45230.0  
